# Modelo Entidad-Relación 

## Librerias 

In [1]:
import numpy as np 
import pandas as pd 
import json 
from sqlalchemy import create_engine 
from sqlalchemy.types import CHAR,VARCHAR, INTEGER,DATE,DATETIME,BOOLEAN,FLOAT,NUMERIC

## Lectura de datos

In [2]:
ruta = '../datos/incidentes_viales_feb_2022.csv'
df = pd.read_csv(ruta, encoding='latin-1', on_bad_lines='skip', dtype=str)
df.head(3)

,folio,fecha_creacion,hora_creacion,dia_semana,fecha_cierre,hora_cierre,incidente_c4,delegacion_inicio,latitud,longitud,ano_cierre,clas_con_f_alarma,tipo_entrada,delegacion_cierre,mes_cierre,codigo_cierre
0,C4/131231/03346,2013-12-31,22:44:53,Martes,2014-01-01,02:11:35,accidente-choque sin lesionados,IZTAPALAPA,19.31469,-99.07113,2014,EMERGENCIA,LLAMADA DEL 066,IZTAPALAPA,Enero,I
1,C4/140101/04189,2014-01-01,21:45:43,MiÃ©rcoles,2014-01-01,23:55:59,accidente-choque con lesionados,ALVARO OBREGON,19.36336,-99.19104,2014,URGENCIAS MEDICAS,LLAMADA DEL 066,ALVARO OBREGON,Enero,A
2,C4/140101/03736,2014-01-01,19:47:28,MiÃ©rcoles,2014-01-01,20:27:26,accidente-choque sin lesionados,TLAHUAC,19.29544,-99.05488,2014,EMERGENCIA,LLAMADA DEL 066,TLAHUAC,Enero,A


## Tratamiento de los datos

In [3]:
df.describe()

,folio,fecha_creacion,hora_creacion,dia_semana,fecha_cierre,hora_cierre,incidente_c4,delegacion_inicio,latitud,longitud,ano_cierre,clas_con_f_alarma,tipo_entrada,delegacion_cierre,mes_cierre,codigo_cierre
count,1640418,1640418,1640418,1640418,1640417,1640416,1640416,1640210,1639973,1639981,1640416,1640416,1640416,1640226,1640414,1640414
unique,1625609,2954,106433,32,2956,107336,34,18,153937,148809,11,5,24,16,13,5
top,VCA/20220128/00001,2022-01-22,20:44:00,Viernes,2022-01-22,21:59:00,accidente-choque sin lesionados,IZTAPALAPA,19.30431996,-99.08024004,2018,EMERGENCIA,LLAMADA DEL 911,IZTAPALAPA,Noviembre,A
freq,2,1344,84,270299,1288,129,880903,268284,2299,2281,246577,929091,965232,269881,168900,693737


- folio

In [4]:
# Duplicados 
df.drop_duplicates('folio',inplace=True)
df = df.reset_index(drop=True)

In [5]:
#Verificamos los datos para localizar errores 
[df['folio'].map(len).min(),df['folio'].map(len).mode()[0],df['folio'].map(len).max()]

[1, 15, 18]

In [6]:
df[df['folio'].map(len) < 15]

,folio,fecha_creacion,hora_creacion,dia_semana,fecha_cierre,hora_cierre,incidente_c4,delegacion_inicio,latitud,longitud,ano_cierre,clas_con_f_alarma,tipo_entrada,delegacion_cierre,mes_cierre,codigo_cierre
992641,,2019-12-21,03:27:18,accidente-choque sin lesionados,TLAHUAC,19.26615996,-99.00128016,2019,EMERGENCIA,LLAMADA DEL 911,TLAHUAC,Diciembre,A,NaN,NaN,NaN
1179217,C5/,2016-12-25,00:42:47,lesionado-atropellado,IZTAPALAPA,19.31706,-99.06513012,2016,URGENCIAS MEDICAS,LLAMADA DEL 066,IZTAPALAPA,Diciembre,A,NaN,NaN,NaN


In [7]:
# En este caso eliminamos los registros con errores ya que no se tiene un folio de registro
df.drop([992641,1179217], axis=0, inplace=True)

In [8]:
# Los folios con longitud mayor a 15 son correctos 
df = df.reset_index(drop=True)
df[df['folio'].map(len) > 15].value_counts()

folio               fecha_creacion  hora_creacion  dia_semana  fecha_cierre  hora_cierre  incidente_c4                     delegacion_inicio  latitud    longitud    ano_cierre  clas_con_f_alarma  tipo_entrada     delegacion_cierre  mes_cierre  codigo_cierre
AOB/20210910/00106  2021-09-10      14:41:25       Viernes     2021-09-10    15:55:06     accidente-volcadura              ALVARO OBREGON     19.398067  -99.198987  2021        EMERGENCIA         RADIO            ALVARO OBREGON     Septiembre  F                1
C5/20211018/00911   2021-10-18      08:24:29       Lunes       2021-10-18    08:50:23     lesionado-atropellado            ALVARO OBREGON     19.33913   -99.232826  2021        URGENCIAS MEDICAS  LLAMADA DEL 911  ALVARO OBREGON     Octubre     D                1
C5/20211018/00878   2021-10-18      08:05:37       Lunes       2021-10-18    09:50:37     accidente-motociclista           IZTAPALAPA         19.391267  -99.046337  2021        URGENCIAS MEDICAS  LLAMADA DEL 911  IZT

-  hora_creacion

In [9]:
df['hora_creacion'].map(len).value_counts()

8     1596819
7       26781
11       1833
10        156
9          16
6           1
4           1
Name: hora_creacion, dtype: int64

In [10]:
# Se convierten los datos a datetime y los errores se manejan con NaN
df['hora_creacion'] = pd.to_datetime(df['hora_creacion'], errors='coerce')

# Nos quedamos sólo con la hora
df['hora_creacion'] = df['hora_creacion'].dt.strftime('%H:%M:%S')

# Para está practica usaremos '00:00:00' para las horas faltantes 
df['hora_creacion'].fillna('00:00:00', inplace=True)

In [11]:
# Verificación 
df['hora_creacion'].map(len).value_counts()

8    1625607
Name: hora_creacion, dtype: int64

- fecha_creacion

In [12]:
# Todos los registros tienen correcto el dato 
df['fecha_creacion'].map(len).value_counts()

10    1625607
Name: fecha_creacion, dtype: int64

In [13]:
# 'fecha_creacion' se concatenara con 'hora_creacion' para tener un sólo registro 
df['fecha_creacion'] = df['fecha_creacion']+' '+ df['hora_creacion']
df.drop(['hora_creacion'],axis=1, inplace = True)
df['fecha_creacion'] = pd.to_datetime(df['fecha_creacion'])
df.head(3)

,folio,fecha_creacion,dia_semana,fecha_cierre,hora_cierre,incidente_c4,delegacion_inicio,latitud,longitud,ano_cierre,clas_con_f_alarma,tipo_entrada,delegacion_cierre,mes_cierre,codigo_cierre
0,C4/131231/03346,2013-12-31 22:44:53,Martes,2014-01-01,02:11:35,accidente-choque sin lesionados,IZTAPALAPA,19.31469,-99.07113,2014,EMERGENCIA,LLAMADA DEL 066,IZTAPALAPA,Enero,I
1,C4/140101/04189,2014-01-01 21:45:43,MiÃ©rcoles,2014-01-01,23:55:59,accidente-choque con lesionados,ALVARO OBREGON,19.36336,-99.19104,2014,URGENCIAS MEDICAS,LLAMADA DEL 066,ALVARO OBREGON,Enero,A
2,C4/140101/03736,2014-01-01 19:47:28,MiÃ©rcoles,2014-01-01,20:27:26,accidente-choque sin lesionados,TLAHUAC,19.29544,-99.05488,2014,EMERGENCIA,LLAMADA DEL 066,TLAHUAC,Enero,A


- hora_cierre

In [14]:
df['hora_cierre'] = df['hora_cierre'].astype(str)
df['hora_cierre'].map(len).value_counts()

8     1596009
7       27597
11       1829
10        148
9          19
6           3
3           2
Name: hora_cierre, dtype: int64

In [15]:
# De igual manera que con 'hora_creacion' se trabajan los datos de la misma manera 

df['hora_cierre'] = pd.to_datetime(df['hora_cierre'], errors='coerce')
df['hora_cierre'] = df['hora_cierre'].dt.strftime('%H:%M:%S')
df['hora_cierre'].fillna('00:00:00', inplace=True)

In [16]:
# Verificación 
df['hora_cierre'].map(len).value_counts()

8    1625607
Name: hora_cierre, dtype: int64

- fecha cierre

In [17]:
df['fecha_cierre'] = df['fecha_cierre'].astype(str)
df['fecha_cierre'].map(len).value_counts()


10    1625605
1           1
3           1
Name: fecha_cierre, dtype: int64

In [18]:
# Registros sin datos
df[df['fecha_cierre'].map(len)!=10]

,folio,fecha_creacion,dia_semana,fecha_cierre,hora_cierre,incidente_c4,delegacion_inicio,latitud,longitud,ano_cierre,clas_con_f_alarma,tipo_entrada,delegacion_cierre,mes_cierre,codigo_cierre
992640,C5/191221/01266,2019-12-21 02:30:05,MiÃ©rcoles,A,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1179215,C5/161224/06130,2016-12-24 19:54:31,MiÃ©rcoles,nan,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Eliminamos los registros con errores 
df.drop([992640,1179215], axis=0, inplace=True)

In [20]:
# De igual manera concatenamos 'fecha_cierre' y 'hora_cierre'
df['fecha_cierre'] = df['fecha_cierre']+' '+ df['hora_cierre']
df.drop(['hora_cierre'],axis=1, inplace = True)
df['fecha_cierre'] = pd.to_datetime(df['fecha_cierre'])
df.head(3)

,folio,fecha_creacion,dia_semana,fecha_cierre,incidente_c4,delegacion_inicio,latitud,longitud,ano_cierre,clas_con_f_alarma,tipo_entrada,delegacion_cierre,mes_cierre,codigo_cierre
0,C4/131231/03346,2013-12-31 22:44:53,Martes,2014-01-01 02:11:35,accidente-choque sin lesionados,IZTAPALAPA,19.31469,-99.07113,2014,EMERGENCIA,LLAMADA DEL 066,IZTAPALAPA,Enero,I
1,C4/140101/04189,2014-01-01 21:45:43,MiÃ©rcoles,2014-01-01 23:55:59,accidente-choque con lesionados,ALVARO OBREGON,19.36336,-99.19104,2014,URGENCIAS MEDICAS,LLAMADA DEL 066,ALVARO OBREGON,Enero,A
2,C4/140101/03736,2014-01-01 19:47:28,MiÃ©rcoles,2014-01-01 20:27:26,accidente-choque sin lesionados,TLAHUAC,19.29544,-99.05488,2014,EMERGENCIA,LLAMADA DEL 066,TLAHUAC,Enero,A


- dia_semana

In [21]:
# Cómo existen días con errores, nos ayudaremos de la columna 'fecha_creacion' para obtener el día correcto 
df['dia_semana'] = df['fecha_creacion'].dt.strftime('%A')

# Reemplazamos los días en un formato más conveniente
df['dia_semana'] = df['dia_semana'].replace({
    'Monday': 'LUN',
    'Tuesday': 'MAR',
    'Wednesday': 'MIE',
    'Thursday': 'JUE',
    'Friday': 'VIE',
    'Saturday':'SAB',
    'Sunday':'DOM'
})

- incidente_c4

In [22]:
df[['incidente_c4']].drop_duplicates().dropna().reset_index(drop=True).head(10)

,incidente_c4
0,accidente-choque sin lesionados
1,accidente-choque con lesionados
2,accidente-persona atrapada / desbarrancada
3,lesionado-atropellado
4,lesionado-accidente automovilÃ­stico
5,accidente-volcadura
6,accidente-otros
7,cadÃ¡ver-atropellado
8,detenciÃ³n ciudadana-atropellado
9,accidente-motociclista


In [23]:
# Para corregir los errores en 'incidente_c4' se creo el archivo 'diccionario_incidentes.csv' 
ruta_2 = '../datos/diccionario_incidentes.csv'
incidentes = pd.read_csv(ruta_2, dtype=str)
incidentes.head(10)

,incidente_c4,incidente
0,accidente-choque sin lesionados,Accidente – Choque sin lesionados
1,accidente-choque con lesionados,Accidente – Choque con lesionados
2,accidente-persona atrapada / desbarrancada,Accidente – Persona atrapada/ desbarrancada
3,lesionado-atropellado,Lesionado - Atropellado
4,lesionado-accidente automovilÃ­stico,Lesionado - Accidente automovilístico
5,accidente-volcadura,Accidente – Volcadura
6,accidente-otros,Accidente – Otros
7,cadÃ¡ver-atropellado,Cadáver – Atropellado
8,detenciÃ³n ciudadana-atropellado,Detención ciudadana – Atropellado
9,accidente-motociclista,Accidente – Motociclista


In [24]:
# Creación de diccionario 
incid_dic = dict(zip(incidentes['incidente_c4'], incidentes['incidente']))

# Corrección 
df['incidente_c4'] = df['incidente_c4'].replace(incid_dic)

- delegacion_inicio

In [25]:
df['delegacion_inicio'].value_counts()

IZTAPALAPA             265599
GUSTAVO A. MADERO      171038
CUAUHTEMOC             165319
MIGUEL HIDALGO         130276
COYOACAN               126869
BENITO JUAREZ          120958
ALVARO OBREGON         118376
VENUSTIANO CARRANZA    112541
TLALPAN                105903
AZCAPOTZALCO            71463
IZTACALCO               70137
XOCHIMILCO              62800
TLAHUAC                 40838
CUAJIMALPA              31132
MAGDALENA CONTRERAS     23471
MILPA ALTA               8682
Name: delegacion_inicio, dtype: int64

- latitud y longitud 

In [26]:
# Convertimos al tipo de dato correcto y se tratan los errores 
df['latitud'] = pd.to_numeric(df['latitud'], errors='coerce')
df['longitud'] = pd.to_numeric(df['longitud'], errors='coerce')

In [27]:
# Redondeamos a 6 decimales puesto que estamos trabajando con latitud y longitud
df['latitud'] = df['latitud'].apply(lambda x: round(x, 6) if not pd.isna(x) else np.nan)
df['longitud'] = df['longitud'].apply(lambda x: round(x, 6) if not pd.isna(x) else np.nan)

- ano_cierre

In [28]:
df['ano_cierre'].value_counts()

2018    246577
2019    229463
2017    226533
2021    198472
2016    190994
2015    184830
2014    171242
2020    162685
2022     14809
Name: ano_cierre, dtype: int64

- clas_con_f_alarma

In [29]:
df['clas_con_f_alarma'].value_counts()

EMERGENCIA           921998
URGENCIAS MEDICAS    691688
FALSA ALARMA          10879
DELITO                 1040
Name: clas_con_f_alarma, dtype: int64

- tipo_entrada

In [30]:
df['tipo_entrada'].value_counts()

LLAMADA DEL 911         952361
LLAMADA DEL 066         463430
BOTÃN DE AUXILIO        92556
RADIO                    90627
LLAMADA APP911            9895
ZELLO                     6451
REDES                     5528
CÃMARA                   3993
APLICATIVOS                583
SÃ¡bado                     82
MiÃ©rcoles                  42
MiÃ©rco                     33
cadÃ¡ver-atropellado         6
cadÃ¡ver-atropell            4
CÃ¡bado                      3
CÃMARo                      2
CÃMAdo                      2
CiÃ©rco                      2
CÃ©rco                      1
BOTÃN DE tropell            1
cadÃ¡verE AUXILIO            1
CÃMado                      1
MiÃ©rcol1s                   1
Name: tipo_entrada, dtype: int64

In [31]:
# Se creo el archivo alarma.csv para los errores 
ruta3 = '../datos/alarma.csv'
alarmas = pd.read_csv(ruta3, dtype=str)
alarmas

,tipo_entrada,entrada
0,LLAMADA DEL 066,LLAMADA DEL 066
1,BOTÃN DE AUXILIO,BOTÓN DE AUXILIO
2,RADIO,RADIO
3,CÃMARA,CÁMARA
4,LLAMADA DEL 911,LLAMADA DEL 911
5,REDES,REDES
6,LLAMADA APP911,LLAMADA DEL 911
7,ZELLO,ZELLO
8,MiÃ©rcoles,NaN
9,APLICATIVOS,APLICATIVOS


In [32]:
# Diccionario con correcciones 
dic_alarma = dict(zip(alarmas['tipo_entrada'],alarmas['entrada']))

# Corrección de los datos 
df['tipo_entrada'] = df['tipo_entrada'].replace(dic_alarma)
df['tipo_entrada'].value_counts()

LLAMADA DEL 911     962256
LLAMADA DEL 066     463430
BOTÓN DE AUXILIO     92556
RADIO                90627
ZELLO                 6451
REDES                 5528
CÁMARA                3993
APLICATIVOS            583
Name: tipo_entrada, dtype: int64

## Entidades 

### Entidad Delegación 

In [33]:
ent_delegacion = df[['delegacion_inicio']].drop_duplicates().dropna().reset_index(drop=True)
ent_delegacion.insert(0,'id',ent_delegacion.index+1)
ent_delegacion.columns = ['id_del','delegacion']
ent_delegacion

,id_del,delegacion
0,1,IZTAPALAPA
1,2,ALVARO OBREGON
2,3,TLAHUAC
3,4,COYOACAN
4,5,CUAUHTEMOC
5,6,MAGDALENA CONTRERAS
6,7,GUSTAVO A. MADERO
7,8,VENUSTIANO CARRANZA
8,9,TLALPAN
9,10,IZTACALCO


### Entidad Incidente

In [34]:
ent_incidente = df[['incidente_c4']].drop_duplicates().dropna().reset_index(drop=True)
ent_incidente.insert(0,'id_incid',ent_incidente.index+1)
ent_incidente.columns = ['id_incid','incidente']
ent_incidente

,id_incid,incidente
0,1,Accidente – Choque sin lesionados
1,2,Accidente – Choque con lesionados
2,3,Accidente – Persona atrapada/ desbarrancada
3,4,Lesionado - Atropellado
4,5,Lesionado - Accidente automovilístico
5,6,Accidente – Volcadura
6,7,Accidente – Otros
7,8,Cadáver – Atropellado
8,9,Detención ciudadana – Atropellado
9,10,Accidente – Motociclista


### Entidad Alarma

In [35]:
ent_alarma = df[['clas_con_f_alarma']].drop_duplicates().dropna().reset_index(drop=True)
ent_alarma.insert(0,'id_alarma',ent_alarma.index+1)
ent_alarma.columns =  ['id_alarma','alarma']
ent_alarma

,id_alarma,alarma
0,1,EMERGENCIA
1,2,URGENCIAS MEDICAS
2,3,FALSA ALARMA
3,4,DELITO


### Entidad Entrada

In [36]:
ent_entrada = df[['tipo_entrada']].drop_duplicates().dropna().reset_index(drop=True)
ent_entrada.insert(0,'id_entrada',ent_entrada.index+1)
ent_entrada.columns = ['id_entrada','entrada']
ent_entrada

,id_entrada,entrada
0,1,LLAMADA DEL 066
1,2,BOTÓN DE AUXILIO
2,3,RADIO
3,4,CÁMARA
4,5,LLAMADA DEL 911
5,6,REDES
6,7,ZELLO
7,8,APLICATIVOS


### Entidad Localización 

In [37]:
ent_localizacion = df[['latitud','longitud']].drop_duplicates().dropna().reset_index(drop=True)
ent_localizacion.insert(0,'id',ent_localizacion.index + 1)
ent_localizacion.columns = ['id_loc','lat','lon']
ent_localizacion

,id_loc,lat,lon
0,1,19.314690,-99.071130
1,2,19.363360,-99.191040
2,3,19.295440,-99.054880
3,4,19.326870,-99.068880
4,5,19.310150,-99.108050
...,...,...,...
198946,198947,19.375937,-99.123713
198947,198948,19.316638,-99.108991
198948,198949,19.316693,-99.108924
198949,198950,19.405058,-99.145645


### Entidad Registro

In [38]:
ent_registro = df
ent_registro.head(2)

,folio,fecha_creacion,dia_semana,fecha_cierre,incidente_c4,delegacion_inicio,latitud,longitud,ano_cierre,clas_con_f_alarma,tipo_entrada,delegacion_cierre,mes_cierre,codigo_cierre
0,C4/131231/03346,2013-12-31 22:44:53,MAR,2014-01-01 02:11:35,Accidente – Choque sin lesionados,IZTAPALAPA,19.31469,-99.07113,2014,EMERGENCIA,LLAMADA DEL 066,IZTAPALAPA,Enero,I
1,C4/140101/04189,2014-01-01 21:45:43,MIE,2014-01-01 23:55:59,Accidente – Choque con lesionados,ALVARO OBREGON,19.36336,-99.19104,2014,URGENCIAS MEDICAS,LLAMADA DEL 066,ALVARO OBREGON,Enero,A


In [39]:
# Cruces de tablas 
ent_registro = ent_registro.merge(ent_incidente,left_on = 'incidente_c4', right_on='incidente', how="inner")
ent_registro.drop(['incidente_c4','incidente'],axis=1, inplace = True)

In [40]:
# Cruce doble latitud y longitud 
ent_registro = ent_registro.merge(ent_localizacion,
left_on=['latitud','longitud'],
right_on=['lat','lon'],
how='inner').drop(['latitud','longitud','lat','lon'],axis=1)

In [41]:
ent_registro = ent_registro.merge(ent_alarma,left_on = 'clas_con_f_alarma', right_on='alarma', how="inner")
ent_registro.drop(['clas_con_f_alarma','alarma'],axis=1, inplace = True)

In [42]:
ent_registro = ent_registro.merge(ent_entrada,left_on = 'tipo_entrada', right_on='entrada', how="inner")
ent_registro.drop(['tipo_entrada','entrada'],axis=1, inplace = True)

In [43]:
ent_registro = ent_registro.merge(ent_delegacion,left_on = 'delegacion_inicio', right_on='delegacion', how="inner")
ent_registro.drop(['delegacion_inicio','delegacion'],axis=1, inplace = True)
ent_registro.rename(columns={'id_del': 'id_del_inicio'}, inplace=True)

In [44]:
ent_registro = ent_registro.merge(ent_delegacion,left_on = 'delegacion_cierre', right_on='delegacion', how="inner")
ent_registro.drop(['delegacion_cierre','delegacion'],axis=1, inplace = True)
ent_registro.rename(columns={'id_del': 'id_del_cierre'}, inplace=True)

In [45]:
ent_registro = ent_registro[['folio', 'fecha_creacion', 'dia_semana', 'id_incid', 'id_loc', 'id_alarma',
       'id_entrada', 'id_del_inicio', 'fecha_cierre', 'ano_cierre',
       'mes_cierre', 'codigo_cierre', 'id_del_cierre']]
ent_registro

,folio,fecha_creacion,dia_semana,id_incid,id_loc,id_alarma,id_entrada,id_del_inicio,fecha_cierre,ano_cierre,mes_cierre,codigo_cierre,id_del_cierre
0,C4/131231/03346,2013-12-31 22:44:53,MAR,1,1,1,1,1,2014-01-01 02:11:35,2014,Enero,I,1
1,C4/140219/01156,2014-02-19 12:07:07,MIE,1,1,1,1,1,2014-02-19 14:21:51,2014,Febrero,I,1
2,C4/131231/03218,2013-12-31 22:14:37,MAR,1,4,1,1,1,2014-01-01 02:09:56,2014,Enero,I,1
3,C4/131231/03146,2013-12-31 21:58:26,MAR,1,4,1,1,1,2014-01-01 07:51:10,2014,Enero,I,1
4,C4/150630/02213,2015-06-30 14:07:03,MAR,1,4,1,1,1,2015-07-01 00:30:32,2015,Julio,A,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1624770,C5/190810/08630,2019-08-10 20:39:36,SAB,2,10577,2,5,4,2019-08-10 20:53:32,2019,Agosto,D,6
1624771,AO/200512/00740,2020-05-12 05:45:11,MAR,1,25979,1,3,4,2020-05-12 10:56:10,2020,Mayo,A,6
1624772,C5/200913/06133,2020-09-13 15:57:00,DOM,16,77895,1,5,16,2020-09-13 19:25:00,2020,Septiembre,N,6
1624773,C5/190504/08178,2019-05-04 19:36:05,SAB,14,40744,1,3,16,2019-05-04 21:48:20,2019,Mayo,A,6


# Carga de datos a MySQL

In [46]:
# Credenciales 
creds = json.load(open('../../../json/creds_incidentes.json','rb'))

In [47]:
# Conexión 
cnx = create_engine(f'mysql+pymysql://{creds["u"]}:{creds["p"]}@{creds["h"]}/{creds["b"]}')

In [54]:
# Entidad delegación 
ent_delegacion.to_sql(name='tbl_delegacion',index=False,con=cnx,if_exists='replace',
dtype=dict(zip(ent_delegacion.columns,[INTEGER,VARCHAR(19)])))

16

In [55]:
# Entidad indidente
ent_incidente.to_sql(name='tbl_incidente',index=False,con=cnx,if_exists='replace',
dtype=dict(zip(ent_incidente.columns,[INTEGER,VARCHAR(47)])))

27

In [59]:
# Entidad alarma
ent_alarma.to_sql(name='tbl_alarma',index=False,con=cnx,if_exists='replace',
dtype=dict(zip(ent_alarma.columns,[INTEGER,VARCHAR(17)])))

4

In [61]:
# Entidad entrada 
ent_entrada.to_sql(name='tbl_entrada',index=False,con=cnx,if_exists='replace',
dtype=dict(zip(ent_entrada.columns,[INTEGER,VARCHAR(16)])))

8

In [62]:
# Entidad localización 
ent_localizacion.to_sql(name='tbl_localizacion',index=False,con=cnx,if_exists='replace',
dtype=dict(zip(ent_localizacion.columns,[INTEGER,FLOAT,FLOAT])))

198951

In [72]:
# Entidad registro 
ent_registro.to_sql(name='tbl_registro',index=False,con=cnx,if_exists='replace', chunksize=100000, 
dtype=dict(zip(ent_registro.columns,[VARCHAR(18),DATETIME,CHAR(3),INTEGER,INTEGER,INTEGER,INTEGER,INTEGER,
                                     DATETIME,NUMERIC,VARCHAR(10),CHAR(1),INTEGER])))

1624775